In [1]:
import math
import torch
import pandas as pd
from operator import itemgetter
import random
from pathlib import Path
from torch.utils.data import Dataset, DataLoader

In [462]:
# also maybe use named_tuples to represent the (h,r,t,h',t')

# if torch.cuda.is_available():  
#   dev = "cuda:0" 
# else:  
#   dev = "cpu"
# convert code to use cuda in the end 
# https://medium.com/ai%C2%B3-theory-practice-business/use-gpu-in-your-pytorch-code-676a67faed09
# torch.cuda.is_available()

In [2]:
# possibly try using multiprocessing (as of now didn't work)
# https://www.youtube.com/results?search_query=concurrency+python
# https://www.realpythonproject.com/how-to-parallelize-for-loops-in-python-and-work-with-shared-dictionaries/

def generate_false_samples_fast(entities_dict, sample, head_or_tail, training_data, index_start_of_relation, allow_shift):
    '''
    
    '''
    running = True
    counter = 0

    # 1 for head
    if head_or_tail:
        # generate false head entity
        # to break in case our tail entity has a relation with every other entity 
        while(running):
            unique = True
            random_entity = random.choice(entities_dict)
            for i in range(index_start_of_relation, len(training_data[index_start_of_relation:])):
                if random_entity == training_data[i][0] and sample[2] == training_data[i][2]:
                    unique = False
                    running = False
                    break
                if training_data[i][1] != training_data[index_start_of_relation][1]:
                    running = False
                    break
                    
            counter += 1
            if unique or counter == 100:
                break
        if allow_shift and not unique:
            sample = generate_false_samples_fast(entities_dict, sample, 0, training_data, index_start_of_relation, False)

        elif unique:
            sample.append(random_entity)
            sample.append(sample[2])
        else:
            print("mistake")
            
    # 0 for tail
    else:
        # generate false tail entity
        while(running):
            random_entity = random.choice(entities_dict)
            unique = True
            for i in range(index_start_of_relation, len(training_data[index_start_of_relation:])):
                if sample[0] == training_data[i][0] and random_entity == training_data[i][2]:
                    unique = False
                    running = False
                    break
                    
                if training_data[i][1] != training_data[index_start_of_relation][1]:
                    running = False
                    break
            counter += 1
            if unique or counter == 100:
                break
        
        if allow_shift and not unique:
            sample = generate_false_samples_fast(entities_dict, sample, 1, training_data, index_start_of_relation, False) 
        elif unique:
            sample.append(sample[0]) 
            sample.append(random_entity)
        else:
            print("mistake")
    return sample

In [3]:
def read_triplets_file(path, header, seperator):
    return pd.read_csv(path, header= header, sep=seperator)

In [38]:
def read_training_data(path, header, seperator):
    '''
    returns a list of sorted (according to relation) training instances 
    '''
    data = pd.read_csv(path, header=header, sep=seperator)
    training_data = data.values.tolist()
# #     print(training_data[0:10])
    training_data = sorted(training_data, key=itemgetter(1)) 
    return training_data

In [42]:
# chose the dimension to be d=20000
def convert_to_dict(text_data):
    output_dict = dict()
#     torch.split(temp1, 1, dim=1)    
    matrix=torch.rand(20000, len(text_data))
    vectors = torch.split(matrix, 1, dim=1)
    
    for i in range(len(text_data)):
#         temp = torch.reshape(vectors[i][0], (-1, 1))
        output_dict[text_data[i]] = vectors[i]/torch.linalg.vector_norm(vectors[i])
        output_dict[text_data[i]].requires_grad=True
    return output_dict

In [6]:
def false_samples_generated():
    extended_data = Path("./data/wn18rr/extended_training_data.txt")
    return extended_data.is_file()

In [7]:
def encoding_training_data(entity_dict, relation_dict, training_data):
    for i in range(len(training_data)):
        training_data[i][0] = entity_dict[training_data[i][0]]
        training_data[i][1] = relation_dict[training_data[i][1]]
        training_data[i][2] = entity_dict[training_data[i][2]]
        training_data[i][3] = entity_dict[training_data[i][3]]
        training_data[i][4] = entity_dict[training_data[i][4]]
    return training_data

In [8]:
def scoring_function(head, relation, tail):
    score = - torch.linalg.vector_norm(head + relation - tail)
    return score

In [9]:
# sample = (h, r, t, h', t')
def loss_func(head, relation, tail, false_head, false_tail, margin):
    loss = max(0, margin - scoring_function(head, relation, tail) + scoring_function(false_head, relation, false_tail))
    return loss

In [35]:
entities_temp_list = read_triplets_file('./data/wn18rr/entities.dict', header=None, seperator='\t')[1]
entities = convert_to_dict(entities_temp_list)

In [13]:
# len(entities[260881])
# entities

In [14]:
# entities_temp_list = pd.read_csv('./data/wn18rr/entities.dict', header=None, sep='\t')[1]
# entities_matrix = torch.eye(len(entities_temp_list))
# entities_vectors = torch.split(entities_matrix, 1)
# entities = dict()
# for i in range(len(entities_temp_list)):
#     entities[entities_temp_list[i]] = torch.reshape(entities_vectors[i][0], (-1, 1))
#     entities[entities_temp_list[i]].requires_grad=True

In [39]:
relation_temp_list = read_triplets_file('./data/wn18rr/relations.dict', header=None, seperator='\t')[1]
relations = convert_to_dict(relation_temp_list)
# relations

In [401]:
# len(relations)

In [402]:
# relation_temp_list = pd.read_csv('./data/wn18rr/relations.dict', header=None, sep='\t')[1]
# relation_matrix = torch.eye(len(relation_temp_list))
# relation_vectors = torch.split(relation_matrix, 1
# relations = dict()
# for i in range(len(relation_temp_list)):
#     relations[relation_temp_list[i]] = torch.reshape(relation_vectors[i][0], (-1, 1))
#     relations[relation_temp_list[i]].requires_grad=True

In [40]:
training_data = read_training_data('./data/wn18rr/train.txt', header=None, seperator='\t')
training_data[0:10]

[[1698271, '_also_see', 1754576],
 [452512, '_also_see', 1695567],
 [1718867, '_also_see', 612114],
 [631391, '_also_see', 1878466],
 [931555, '_also_see', 604617],
 [1672607, '_also_see', 485711],
 [1725712, '_also_see', 1256332],
 [1835496, '_also_see', 781000],
 [503982, '_also_see', 889831],
 [2531422, '_also_see', 1076793]]

In [43]:
if false_samples_generated():
    print("found file")
    read_training_data = pd.read_csv('./data/wn18rr/extended_training_data.txt', header=None, sep='\t')
    training_data = read_training_data.values.tolist()
else:
    for i  in range(len(training_data)):
        print(i)
        if not training_data[i][1] == training_data[i-1][1]:
            index_start_of_relation = i
            print("new index: ", index_start_of_relation)
            print(training_data[i][1])
        head_or_tail = random.randint(0, 1)
        training_data[i] = generate_false_samples_fast(entities_temp_list, 
                                              training_data[i], 
                                              head_or_tail, 
                                              training_data, 
                                              index_start_of_relation,
                                              True)
    
    textfile = open("./data/wn18rr/extended_training_data.txt", "w")
    for row in training_data:
        if len(row) == 5:
            textfile.write(str(row[0]) + "\t" + str(row[1]) + "\t" + str(row[2]) + "\t" + str(row[3]) +"\t" + str(row[4]) + "\n")
        elif len(row) == 3:
            textfile.write(str(row[0]) + "\t" + str(row[1]) + "\t" + str(row[2]) + "\n")
    textfile.close()

found file


In [44]:
# for i  in range(len(temp_training_data)):
#     if not temp_training_data[i][1] == temp_training_data[i-1][1]:
#         index_start_of_relation = i
#         print("new index: ", index_start_of_relation)
#         print(training_data[i][1])
        

# counter = 0
# for i in training_data:
#     if len(i) != 5:
#         counter +=1
# #         print(len(i))
# print(counter)
print(training_data[0])

[1698271, '_also_see', 1754576, 1698271, 9559404]


In [45]:
# we get the embeddings for (h, r, t# training_data)
training_data = encoding_training_data(entities, relations, training_data)

In [46]:
print(len(training_data[0]))
print(len(training_data[0][0]))
# print(len(training_data[0][1]))
# print(len(training_data[0][2]))
# print(len(training_data[0][3]))
# print(len(training_data[0][4]))
# training_data[0]

# counter = 0
# for i in training_data:
#     if len(i) != 5:
#         counter +=1
#         print(len(i))
# print(counter)

5
20000


In [47]:
# # margin = 1, dimension d = 10000

# train_loader = DataLoader(dataset=training_data,
#                           batch_size=50,
#                           shuffle=True,
#                           num_workers=0)

# # convert to an iterator and look at one random sample
# dataiter = iter(train_loader)
# data = dataiter.next()
# head, relation, tail, false_head, false_tail = data
# # print(head, "\n", relation)

# # # Dummy Training loop
# num_epochs = 5
# total_samples = len(training_data)
# n_iterations = math.ceil(total_samples/4)
# # print(total_samples, n_iterations)
# for epoch in range(num_epochs):
# #     for i, (inputs, labels) in enumerate(train_loader):
#     for i, (head, relation, tail, false_head, false_tail) in enumerate(train_loader):
        
#         # here: 178 samples, batch_size = 4, n_iters=178/4=44.5 -> 45 iterations
#         # Run your training process
# #         print(head.data)
#         # loss
#         l = loss_func(head, relation, tail, false_head, false_tail, 1)
# #         l.retain_grad()
#         head.retain_grad()
#         relation.retain_grad()
#         tail.retain_grad()
#         false_head.retain_grad() 
#         false_tail.retain_grad()
#         # calculate gradients = backward pass
#         l.backward()
        
        
#         with torch.no_grad():
#             head -= 0.1 * head.grad
#             relation -= 0.1 * relation.grad
#             tail -= 0.1 * tail.grad
#             false_head -= 0.1 * false_head.grad
#             false_tail -= 0.1 * false_tail.grad
            
#             # don't forget to zero the gradients
       
        
#         head.grad.zero_()
#         relation.grad.zero_()
#         tail.grad.zero_()
#         false_head.grad.zero_() 
#         false_tail.grad.zero_() 
        
#         if i%100 == 0:
#             print(f'Iteration: {i} | Loss {l.item()}')










# results: 
# Iteration: 0 | Loss 1.0015830993652344
# Iteration: 100 | Loss 0.9973239898681641
# Iteration: 200 | Loss 0.9993381500244141
# Iteration: 300 | Loss 0.9902715682983398
# Iteration: 400 | Loss 1.0056066513061523
# Iteration: 500 | Loss 0.9979467391967773
# Iteration: 600 | Loss 1.0004215240478516
# Iteration: 700 | Loss 0.9965410232543945

In [ ]:
# margin = 1, dimension d = 20000
train_loader = DataLoader(dataset=training_data,
                          batch_size=50,
                          shuffle=True,
                          num_workers=0)

# convert to an iterator and look at one random sample
dataiter = iter(train_loader)
data = dataiter.next()
head, relation, tail, false_head, false_tail = data
# print(head, "\n", relation)

# # Dummy Training loop
num_epochs = 5
total_samples = len(training_data)
n_iterations = math.ceil(total_samples/4)
# print(total_samples, n_iterations)
for epoch in range(num_epochs):
#     for i, (inputs, labels) in enumerate(train_loader):
    for i, (head, relation, tail, false_head, false_tail) in enumerate(train_loader):
        
        # here: 178 samples, batch_size = 4, n_iters=178/4=44.5 -> 45 iterations
        # Run your training process
#         print(head.data)
        # loss
        l = loss_func(head, relation, tail, false_head, false_tail, 0.05)
#         l.retain_grad()
        head.retain_grad()
        relation.retain_grad()
        tail.retain_grad()
        false_head.retain_grad() 
        false_tail.retain_grad()
        # calculate gradients = backward pass
        l.backward()
        
        
        with torch.no_grad():
            head -= 0.1 * head.grad
            relation -= 0.1 * relation.grad
            tail -= 0.1 * tail.grad
            false_head -= 0.1 * false_head.grad
            false_tail -= 0.1 * false_tail.grad
            
            # don't forget to zero the gradients
       
        
        head.grad.zero_()
        relation.grad.zero_()
        tail.grad.zero_()
        false_head.grad.zero_() 
        false_tail.grad.zero_() 
        
        if i%100 == 0:
            print(f'Iteration: {i} | Loss {l.item()}')


Iteration: 0 | Loss 0.05202293395996094
Iteration: 100 | Loss 0.04584026336669922
Iteration: 200 | Loss 0.048508644104003906
Iteration: 300 | Loss 0.07849693298339844
Iteration: 400 | Loss 0.051293373107910156
Iteration: 500 | Loss 0.04837322235107422
Iteration: 600 | Loss 0.05049324035644531
Iteration: 700 | Loss 0.04554939270019531
